### Import Libraries

In [11]:
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict
import folium
from dotenv import dotenv_values

config = dotenv_values("../.env") 

if "GEOAPIFY_KEY" in config.keys():
    API_KEY = config["GEOAPIFY_KEY"]
else:
    raise ValueError("Missing GEOAPIFY_KEY in config, please provide API key!")

### Load Dataset

In [3]:
df = pd.read_csv("../data/instructors_countries.csv")
df.sample(3)

,instructor,countries
113,Jeroen Wilms,"Croatia, Netherlands"
147,Olga Martinelli,Italy
137,Peter Gaertner,"Germany, Italy"


### Determine Lat/Long of Countries

In [29]:
countries_lat_long = dict.fromkeys(", ".join([x for x in list(df['countries'])]).split(", "))

for country in countries_lat_long.keys(): 
    url = f"https://api.geoapify.com/v1/geocode/search?text={country}&apiKey={API_KEY}"

    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    resp = requests.get(url, headers=headers)

    lat = resp.json().get('features')[0].get("properties").get("lat")
    lon = resp.json().get('features')[0].get("properties").get("lon")
    countries_lat_long.update({country: [lat, lon]})

### Display Instructors by Countries

In [47]:
# Transform data to fit markers
instructor_to_country_map = {}

for _, row in df.iterrows():
    name, countries = row
    countries = countries.split(", ")
    
    for country in countries:
        if country not in instructor_to_country_map.keys():
            instructor_to_country_map.update({country: [name]})
        else:
            value = instructor_to_country_map[country]
            value.append(name)
            instructor_to_country_map.update({country: value})
instructor_to_country_map     

{'Egypt': ['Ahmed Hagi',
  'Andrei Dmitriev',
  'Mahmoud Esmat',
  'Niccolo Crespi',
  'Sameh Sokar'],
 'Italy': ['Ahmed Hagi',
  'Niccolo Crespi',
  'Alessandro Vezzani',
  'Mauro Bordignon',
  'Francesco Cameli',
  'Mario Arena',
  'Mattia Popesso',
  'Stefano Gualtieri',
  'Andrea Marassich',
  'Peter Gaertner',
  'Andrea Milani',
  'Bruno Borelli',
  'Caterina De Seta',
  'Claudio Provenzani',
  'Fabio Portella',
  'Ivan Wagner',
  'Jordan Allured',
  'Marco Colman',
  'Olga Martinelli',
  'Romano Rampazzo',
  'Stefano Di Cosimo'],
 'Russia': ['Andrei Dmitriev', 'Kirill A. Egorov', 'Sergey Porkhachev'],
 'South Africa': ['Sameh Sokar'],
 'Argentina': ['Belen Andres-Garcia',
  'Guillermo Riesco',
  'Raul Alvarez Garcia'],
 'Spain': ['Belen Andres-Garcia',
  'Raul Alvarez Garcia',
  'Ricardo Constantino',
  'Julio Cesar Cepeda Vargas',
  'Lu Chen',
  'Nuno Padrao'],
 'Chile': ['Guillermo Riesco'],
 'Brazil': ['Ricardo Constantino', 'Sergio R Schirato'],
 'United States': ['Sergio R S

In [51]:
m = folium.Map(location=[0, 0], zoom_start=4)

# Add markers with custom icons and pop-up information
for key, value in instructor_to_country_map.items():
    lat_long = countries_lat_long.get(key)
    print(key, lat_long, value)
    folium.Marker(lat_long, popup=", ".join(value)).add_to(m)

# Save the map to an HTML file
m.save("instructors spread.html")

Egypt [26.2540493, 29.2675469] ['Ahmed Hagi', 'Andrei Dmitriev', 'Mahmoud Esmat', 'Niccolo Crespi', 'Sameh Sokar']
Italy [42.6384261, 12.674297] ['Ahmed Hagi', 'Niccolo Crespi', 'Alessandro Vezzani', 'Mauro Bordignon', 'Francesco Cameli', 'Mario Arena', 'Mattia Popesso', 'Stefano Gualtieri', 'Andrea Marassich', 'Peter Gaertner', 'Andrea Milani', 'Bruno Borelli', 'Caterina De Seta', 'Claudio Provenzani', 'Fabio Portella', 'Ivan Wagner', 'Jordan Allured', 'Marco Colman', 'Olga Martinelli', 'Romano Rampazzo', 'Stefano Di Cosimo']
Russia [64.6863136, 97.7453061] ['Andrei Dmitriev', 'Kirill A. Egorov', 'Sergey Porkhachev']
South Africa [-28.8166236, 24.991639] ['Sameh Sokar']
Argentina [-34.9964963, -64.9672817] ['Belen Andres-Garcia', 'Guillermo Riesco', 'Raul Alvarez Garcia']
Spain [39.3260685, -4.8379791] ['Belen Andres-Garcia', 'Raul Alvarez Garcia', 'Ricardo Constantino', 'Julio Cesar Cepeda Vargas', 'Lu Chen', 'Nuno Padrao']
Chile [-31.7613365, -71.3187697] ['Guillermo Riesco']
Brazil

In [33]:
lat_long

[-24.7761086, 134.755]